In [ ]:
#import necessary modules
#Python >= 3.5
import sys
assert sys.version_info >= (3, 5)

#Scikit-Learn >= 0.20
import sklearn as sk
assert sklearn.__version__ >= "0.20"

#???
from sklearn.model_selection import StratifiedShuffleSplit

#numpy for calculations
import numpy as np
import os

#matplotlib for figures
import matplotlib as mpl
import matplotlib.pyplot as plt
#a magic function that allows inline plotting so figures are rendered in this notebook
%matplotlib inline

#pandas for ...
import pandas as pd
from pandas.plotting import scatter_matrix

#add other needed modules here
#Tensorflow?

## Setup and Feature Statistics

In [ ]:
#define where to save figures for this project
"""
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
"""

In [ ]:
#define a function to load data from the cv file using pandas
#convert to tensors??? how many dimensions? how many tensors/matrices/vectors?

#call function and store dataframe
scores = ...

#sanity check: take a look at the data
#preview first five lines
scores.head()
#print number of columns, column labels, column data types, memory usage, range index, and non-null number of cells in each column
scores.info()
#gives count, mean, standard deviation, min, max, and percentiles (including median)
scores.describe(perncentiles = [0.25, 0.5, 0.75], include = 'all')

In [ ]:
#plot some data statistics for the report

## Creat a Test Set

In [ ]:
#set a seed so the output is the same every run
np.random.seed(...)

In [ ]:
#split data into training and testing and save in variables
#what is the best way to do that for our data set?
#justification needed for Report 1
#look at sklearn function train_test_split()
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
#test size = ???
#random_state = ???
train_set, test_set = ...

#sanity check: look at how many data points are in train/test
len(train_set)
len(test_set)

In [ ]:
#perform checksum/hash sum OR cyclic rendunancy check (CRC) on data
#how to do it???
#see here: https://www.codeproject.com/Articles/1671/CRC32-Generating-a-checksum-for-a-file

In [2]:
#sanity check: preview test and training data
train_set.head()
test_set.head()

NameError: name 'train_set' is not defined

In [ ]:
#perform stratified shufflesplit cross-validator
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html
#compares random and stratified error

## Visualize the Data

In [ ]:
#create scatter plot of data with alpha=???
#add legends, titles, and color

## Visualize Correlations

In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html

#create scatter matrices displaying totals, medians..

## Prepare Data for ML Algorithms

In [ ]:
#drop data labels

#save labels in a separate variable

In [ ]:
# drop rows that have negative values
#                # days before exam > 180
#                # no final MCAT score
#                # exact duplicate values to another row

# sanity check: preview the data

In [ ]:
# add custom attribute: variance between test scores (if multiple test scores)
# NOTE: make sure variance is calculated between different tests, not between subsections of the same test

## Transformation Pipeline

In [ ]:
# allows you to easily apply a list of transforms and fit methods in a given order
from sklearn.pipeline import Pipeline
# standardizes features, which is a requirement for many ML estimators
from sklearn.preprocessing import StandardScaler

# Select and Train a Model

## Training and Evaluating on the Training Set

In [ ]:
# NOTE: Explain the process of model validation
# how did you split the data into training, validation and test sets
# What are the sizes of each set and why did you make such design choice.

from sklearn.linear_model import LinearRegression

# initialize a linear regression object
lin_reg = LinearRegression()
lin_reg.fit("DATA", "LABELS")

# sanity check: test out predictions and compare to labels
print(lin_reg.predict("SOME DATA"))
print(list("SOME LABELS"))

In [ ]:
# loss function
from sklearn.metrics import mean_squared_error

MCAT_predictions = lin_reg.predict("DATA")
lin_rmse = mean_squared_error("LABELS", MCAT_predictions, squared=False)
lin_rmse

## Evaluation with Cross-Validation

In [ ]:
lin_scores = cross_val_score(lin_reg, "DATA", "LABELS",
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)